In [1]:
# Importing necessary libraries
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy.stats import wasserstein_distance
import pandas as pd

# Defining the provided functions and parameters

# Defining the weight function
def omega(t, x, c1, c2):
    sigma = math.sqrt(np.var(X))
    h1 = (c1/math.sqrt(12))*(T ** (-(1/5)))
    h2 = c2*sigma*(T ** (-(1/5)))
    numerator = K_h1(t/T - t / T, h1) * np.prod(K_h2(x - X[t-1], h2))
    denominator = np.sum([K_h1(s/T - s / T, h1) * np.prod(K_h2(x - X[s-1], h2)) for s in range(1, T+1)])
    return numerator / denominator

# Defining the empirical conditional distribution function estimator
def ecdf(x, v):
    weights = np.array([omega(t, x, c1, c2) for t in range(1, T+1)])
    indicator = (Y <= v).astype(int)
    return np.sum(weights * indicator)

# Define uniform kernels for K1
def K_h1(z, h1):
    return 0.5 if abs(z/h1) <= 1 else 0

# Define Gaussian kernels for K2
def K_h2(z, h2):
    return np.exp(-z**2 / (2 * h2**2))

# Generate locally stationary data
np.random.seed(42)
# Define the time-varying parameter
def time_varying_param(t):
    phi = 0.2 + 0.3 * np.sin(0.1 * t/T)
    return phi

# Simulate the time-varying AR(1) model
def simulate_tv_ar(T, param_func):
    epsilon = np.random.normal(0, 1, T)
    y = np.zeros(T)
    
    for t in range(1, T):
        phi = param_func(t)
        y[t] = phi * y[t-1] + epsilon[t]
    
    return y

# Parameters
T = 300  # Length of time series
d = 1  # Number of covariates
c1 = 5
c2 = 1.68
x_test = np.array([0.5])

c1_c2_pairs = [(0.9, 0.3), (1.8, 0.5), (2.6, 0.8), (3.5, 1.1), (4.4, 1.4), (5.2, 1.6), (6.1, 1.9), (7.0, 2.2), (7.8, 2.5), (8.7, 2.7)]

# Running the replication loop
c1_c2_W1 = {}  # Store average W1 for each pair of c1 and c2
c1_c2_W1_individual = {}  # Store individual W1 for each pair of c1 and c2
c1_c2_MSE = {}  # Store MSE for each pair of c1 and c2
c1_c2_MSE_individual = {}  # Store SE for each pair of c1 and c2

# Running the replication loop
results = []

for (c1, c2) in c1_c2_pairs:
    W1_temp_list = []  # Store W1 for each replication
    MSE_temp_list = []  # Store MSE for each replication
    for rep in range(50):
        # Generate locally stationary data
        np.random.seed(42+rep)  # Use different seed for each replication
        Y = simulate_tv_ar(T, time_varying_param)
        X = np.array([Y[t-2] for t in range(1, T+1)])
        
        v_values = np.linspace(Y.min(), Y.max(), T)  # Update v_values based on new Y
        
        # Estimated F for each v
        F_values = [ecdf(x_test, v) for v in v_values]
        
        # True CDF
        data_sorted = np.sort(Y)
        True_CDF = np.arange(1, len(data_sorted) + 1) / len(data_sorted)
        
        # Compute the 1-Wasserstein distance
        W1 = wasserstein_distance(True_CDF, F_values)
        W1_temp_list.append(W1)
        
        # Compute the MSE
        MSE = np.mean((True_CDF - F_values) ** 2)
        MSE_temp_list.append(MSE)
        
    # Storing individual and average 1-Wasserstein distances for this pair of c1 and c2
    c1_c2_W1[(c1, c2)] = np.mean(W1_temp_list)
    c1_c2_W1_individual[(c1, c2)] = W1_temp_list
    
    # Storing individual SE and MSE for this pair of c1 and c2
    c1_c2_MSE[(c1, c2)] = np.mean(MSE_temp_list)
    c1_c2_MSE_individual[(c1, c2)] = MSE_temp_list
    
    # Results for each pair of c1 and c2
    results.append({
        "c1": c1,
        "c2": c2,
        "Average_W1": np.mean(W1_temp_list),
        "MSE": np.mean(MSE_temp_list)
    })
    
    #result_dict = {"c1": c1, "c2": c2}
    #for i, (W1_value, MSE_value) in enumerate(zip(W1_temp_list, MSE_temp_list)):
        #result_dict[f"W1_rep{i + 1}"] = W1_value
        #result_dict[f"SE_rep{i + 1}"] = MSE_value
    #result_dict["Average_W1"] = np.mean(W1_temp_list)
    #result_dict["Average MSE"] = np.mean(MSE_temp_list)
    #results.append(result_dict)

# Convert results to a DataFrame and display
df = pd.DataFrame(results)
print(df)

print("Individual 1-Wasserstein Distances for each (c1, c2): ", c1_c2_W1_individual)
print("Individual SE for each (c1, c2): ", c1_c2_MSE_individual)

    c1   c2  Average_W1       MSE
0  0.9  0.3    0.124044  0.021165
1  1.8  0.5    0.120256  0.019836
2  2.6  0.8    0.118255  0.019079
3  3.5  1.1    0.117525  0.018758
4  4.4  1.4    0.117197  0.018581
5  5.2  1.6    0.117065  0.018496
6  6.1  1.9    0.116903  0.018393
7  7.0  2.2    0.116768  0.018307
8  7.8  2.5    0.116642  0.018237
9  8.7  2.7    0.116565  0.018197
Individual 1-Wasserstein Distances for each (c1, c2):  {(0.9, 0.3): [0.13562176002464352, 0.1111151332113833, 0.13558466703041414, 0.11409429832753659, 0.09946354081796324, 0.126425338865772, 0.16372480014103058, 0.11990189411139332, 0.16872895340517674, 0.15253403351683248, 0.09959767404528679, 0.10321121714951034, 0.13642031272903285, 0.1611927529580468, 0.09881629548959722, 0.10406601325701403, 0.12668672258475258, 0.11740792663982887, 0.10432778524170547, 0.11946886724824748, 0.13688265947477893, 0.12917776121677083, 0.12467692104198777, 0.14189270104956303, 0.11162992471923336, 0.1429458453666881, 0.14781687030506